In [16]:
import numpy as np
import tensorflow as tf
import operator
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import copy
from sklearn.feature_extraction.text import CountVectorizer

def countChange(data):
    return Counter(data.split(" "))
    
X,y=[],[]
f=open("C:/SMSSpamCollection")
data=list(line[:-1].split('\t') for line in f.readlines())

X=(list(data[i][1] for i in range(len(data))))
y=(list(data[i][0] for i in range(len(data))))

X = np.array(X)
y = np.array(y)

target_names=['ham','spam']

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y)


data2=[]

for i in range(len(data)):        #각각의 데이터의 단어 개수
    data[i]=(countChange(X[i].lower()))

data2=Counter(data2)

for i in range(len(data)):   #단어 개수 총합
    data2=(data2+data[i])

sorted_x = sorted(dict(data2).items(),
               key=operator.itemgetter(1),reverse = True)
features=sorted_x[50:]     #상위 50개 추출

vect = CountVectorizer(stop_words=features)
corpus_feature = vect.fit_transform(X).toarray()



In [19]:
x_data = np.array(corpus_feature[:,:1000], dtype = np.float32)
y_data = np.array(y, dtype = np.float32)

X= tf.placeholder(tf.float32)
Y= tf.placeholder(tf.float32)

W1= tf.Variable(tf.random_normal(([1000,500])), name = 'weight1') #1000차원입력 500차원 출력
b1= tf.Variable(tf.random_normal(([500]), name = 'bias1'))
layer1 = tf.sigmoid(tf.matmul(X, W1)+b1)

W2= tf.Variable(tf.random_normal(([500,100])), name = 'weight2') #500차원입력 100차원 출력
b2= tf.Variable(tf.random_normal(([100]), name = 'bias2'))
layer2 = tf.sigmoid(tf.matmul(layer1, W2)+b2)

W3= tf.Variable(tf.random_normal(([100,1])), name = 'weight3') #100차원입력 1차원 출력
b3= tf.Variable(tf.random_normal(([1]), name = 'bias3'))
hypothesis = tf.sigmoid(tf.matmul(layer2, W3) + b3)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1- hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)

predicted = tf.cast(hypothesis > 0.5 , dtype = tf.float32)
accuracy  = tf.reduce_mean(tf.cast(tf.equal(predicted , Y), dtype = tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        sess.run(train, feed_dict={X: x_data, Y:y_data})
        if step % 10 == 0:
            print(step, sess.run(cost,feed_dict={X: x_data, Y:y_data}), sess.run(W1))
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis:\n", h , "\nPredicted:\n", c, "\nAccuracy:\n", a)


0 1.79127 [[ 1.52370393  0.3796941   0.32532313 ..., -1.60951447 -0.81269217
  -0.14498429]
 [-0.39026201  0.69246    -0.68267965 ...,  1.73913777 -1.89444482
  -0.6577183 ]
 [-0.67475593 -0.02496008  1.27836132 ..., -0.3296673   0.70176244
   0.74046731]
 ..., 
 [ 0.5828743   0.29102278 -0.22914812 ..., -0.96589887  1.49151301
  -1.2559576 ]
 [ 0.89130479 -0.22476774  0.11472388 ..., -0.98355186 -0.46387932
   3.16011882]
 [-0.34155402 -1.11681247 -0.37693313 ...,  0.1626863   0.44632006
   0.68515027]]
10 1.77614 [[ 1.52370393  0.3796941   0.32532135 ..., -1.60951567 -0.81269276
  -0.14498429]
 [-0.3902629   0.6924606  -0.68268085 ...,  1.73913777 -1.89444482
  -0.6577183 ]
 [-0.67475593 -0.02496002  1.27836132 ..., -0.3296673   0.70176244
   0.74046731]
 ..., 
 [ 0.5828737   0.29102278 -0.22914827 ..., -0.96589828  1.49151301
  -1.2559576 ]
 [ 0.89130479 -0.22476774  0.11472388 ..., -0.98355186 -0.46387932
   3.16011882]
 [-0.34155402 -1.11681247 -0.37693313 ...,  0.1626863   0.4463